#### Lab 7 
#### Rouge Metrics

In [2]:
!pip install fuzzywuzzy
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
import numpy as np
from fuzzywuzzy import fuzz 
import ipywidgets as widgets
import pprint
from ipywidgets import interact, interact_manual
import re
__PATH__ = "/content/Lab7 - data.csv"

In [3]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [6]:
df = pd.read_csv(__PATH__,sep=";",header=0)
df

,id,updatedDate,publishedDate,title,summary,authors,category,metaData,downloadLink,filePath
0,http://arxiv.org/abs/1407.6950v1,2014-07-24T16:56:39Z,2014-07-24T16:56:39Z,"How,whenAndHowMuchACardDeckIsWellShuffled.pdf",The Thesis Consider The Mixing Of Few 3 4 ...,Benjamin Isac Fargion,cs.DM,"Italian Thesis In Engeenering Computer, 26 Feb...",http://arxiv.org/pdf/1407.6950v1.pdf,"./files/How,whenAndHowMuchACardDeckIsWellShuff..."
1,http://arxiv.org/abs/0907.0618v1,2009-07-03T12:35:10Z,2009-07-03T12:35:10Z,QuantumIsometryGroups.pdf,This Thesis Contains The Formulation And Com...,Jyotishman Bhowmick,math.OA,Thesis,http://arxiv.org/pdf/0907.0618v1.pdf,./files/QuantumIsometryGroups.pdf
2,http://arxiv.org/abs/1806.09601v2,2018-07-14T17:06:27Z,2018-06-25T17:55:59Z,ComputationAndBoundingOfFolkmanNumbers.pdf,Phd Thesis Under The Supervision Of Professo...,Aleksandar Bikov,math.CO,PhD Thesis,http://arxiv.org/pdf/1806.09601v2.pdf,./files/ComputationAndBoundingOfFolkmanNumbers...
3,http://arxiv.org/abs/1905.03014v1,2019-05-08T11:47:34Z,2019-05-08T11:47:34Z,OnChurch'sThesisInCubicalAssemblies.pdf,"We Show That Church's Thesis, The Axiom Stat...","Andrew Swan, Taichi Uemura,",math.LO,0,http://arxiv.org/pdf/1905.03014v1.pdf,./files/OnChurch'sThesisInCubicalAssemblies.pdf
4,http://arxiv.org/abs/1901.04911v1,2019-01-15T16:24:07Z,2019-01-15T16:24:07Z,UnconstrainedChurchTuringThesisCannotPossiblyB...,The Church Turing Thesis Asserts That If A P...,Yuri Gurevich,cs.LO,0,http://arxiv.org/pdf/1901.04911v1.pdf,./files/UnconstrainedChurchTuringThesisCannotP...
...,...,...,...,...,...,...,...,...,...,...
991,http://arxiv.org/abs/0709.0497v1,2007-09-04T18:40:50Z,2007-09-04T18:40:50Z,HardSpectatorInteractionsInBToPiPiAtOrderAlpha...,In The Present Thesis I Discuss The Hard Spe...,Volker Pilipp,hep-ph,PhD thesis,http://arxiv.org/pdf/0709.0497v1.pdf,./files/HardSpectatorInteractionsInBToPiPiAtOr...
992,http://arxiv.org/abs/0709.2936v1,2007-09-18T23:56:17Z,2007-09-18T23:56:17Z,BayesianClassificationAndRegressionWithHighDim...,This Thesis Responds To The Challenges Of Us...,Longhai Li,stat.ML,"PhD Thesis Submitted to University of Toronto,...",http://arxiv.org/pdf/0709.2936v1.pdf,./files/BayesianClassificationAndRegressionWit...
993,http://arxiv.org/abs/0710.1790v1,2007-10-09T15:20:08Z,2007-10-09T15:20:08Z,StaticAndDynamicPropertiesOfHadronicSystemsWit...,This Thesis Has Been Devoted To The Study Of...,J. M. Verde-Velasco,hep-ph,"PhD thesis, Supervisors: E. Hernandez, J. Niev...",http://arxiv.org/pdf/0710.1790v1.pdf,./files/StaticAndDynamicPropertiesOfHadronicSy...
994,http://arxiv.org/abs/0711.0873v2,2007-11-11T19:21:04Z,2007-11-06T14:13:50Z,AConformalApproachToNumericalCalculationsOfAsy...,This Thesis Is Concerned With The Developmen...,Anıl Zenginoğlu,gr-qc,"PhD thesis, Max-Planck Institute for Gravitati...",http://arxiv.org/pdf/0711.0873v2.pdf,./files/AConformalApproachToNumericalCalculati...


#### Preprocessing the title to list of tokens

In [0]:
titles = list(df['title'].apply(
    lambda t : 
        tuple(
            filter(lambda e:not e in stopwords.words('english'),
                map(lambda e:e.lower(),
                       re.findall('([A-Z]{1}[a-z]+)',t.replace('.pdf','')))
                )
            )
        )
    )

In [0]:

res = {}
for title in titles: #for every title in title list
    synsets = {} #new dist with synsets 
    for word in title: #for every word in title 
        synsets[word]=[synset for synset in wn.synsets(word)] #word: 
    res[title] = synsets

In [9]:
res = {}
for title in titles:
    synsets = {}
    for word in title:
      for synset in wn.synsets(word):  
        synsets[word]=synset 
        print(synset)
        #print(synsets[word])
      print("============")
    res[title] = synsets

Streaming output truncated to the last 5000 lines.
Synset('gauge.n.02')
Synset('gauge.n.03')
Synset('gauge.n.04')
Synset('bore.n.03')
Synset('estimate.v.01')
Synset('gauge.v.02')
Synset('gauge.v.03')
Synset('gauge.v.04')
Synset('gauge.v.05')
Synset('gauge.v.06')
Synset('theory.n.01')
Synset('hypothesis.n.02')
Synset('theory.n.03')
Synset('gravity.n.01')
Synset('graveness.n.01')
Synset('gravity.n.03')
Synset('average.v.01')
Synset('average.v.02')
Synset('average.v.03')
Synset('preciseness.n.02')
Synset('cosmology.n.01')
Synset('cosmology.n.02')
Synset('determine.v.01')
Synset('determine.v.02')
Synset('determine.v.03')
Synset('specify.v.02')
Synset('decide.v.01')
Synset('determine.v.06')
Synset('settle.v.03')
Synset('determine.v.08')
Synset('deciding.s.01')
Synset('leishmania.n.01')
Synset('infection.n.01')
Synset('infection.n.02')
Synset('infection.n.03')
Synset('infection.n.04')
Synset('contagion.n.03')
Synset('infection.n.06')
Synset('infection.n.07')
Synset('degree.n.01')
Synset('gra

In [10]:
print(res[title])

{'theoretical': Synset('theoretical.a.02'), 'analysis': Synset('psychoanalysis.n.01'), 'optimization': Synset('optimization.n.01'), 'problems': Synset('trouble.n.01'), 'properties': Synset('property.n.05'), 'random': Synset('random.a.01'), 'sat': Synset('sit.v.10')}


In [11]:
print(res)

{('much', 'card', 'deck', 'well', 'shuffled'): {'much': Synset('much.r.05'), 'card': Synset('card.v.02'), 'deck': Synset('deck.v.03'), 'well': Synset('well.r.13'), 'shuffled': Synset('shuffle.v.03')}, ('quantum', 'isometry', 'groups'): {'quantum': Synset('quantum.n.02'), 'isometry': Synset('isometry.n.04'), 'groups': Synset('group.v.02')}, ('computation', 'bounding', 'folkman', 'numbers'): {'computation': Synset('calculation.n.02'), 'bounding': Synset('bounce.v.01'), 'numbers': Synset('number.v.06')}, ('church', 'thesis', 'cubical', 'assemblies'): {'church': Synset('church.v.01'), 'thesis': Synset('dissertation.n.01'), 'cubical': Synset('cubelike.s.01'), 'assemblies': Synset('assembly.n.06')}, ('unconstrained', 'church', 'turing', 'thesis', 'cannot', 'possibly', 'true'): {'unconstrained': Synset('unconstrained.s.01'), 'church': Synset('church.v.01'), 'turing': Synset('turing.n.01'), 'thesis': Synset('dissertation.n.01'), 'possibly': Synset('possibly.r.02'), 'true': Synset('true.r.01')}

In [0]:
def get_hypernyms(list):
    hypernyms_list = []
    synsets_ = wn.synsets(list) 
    for i in range(len(synsets_)):
      hypernyms_list.extend(wn.synsets(list)[i].hypernyms()) 
    hyper = set(hypernyms_list)
    return hyper

In [0]:
def fscore_(a,b):
    intersection = a.intersection(b)
    if (len(intersection) == 0):
        fscore = 0
    else:
        prec = len(intersection)/(len(a))
        recall = len(intersection)/(len(b))
        fscore = 2 * prec * recall/(prec + recall)
    return intersection, fscore


def distance(a,b):
  a = set(a)
  b = set(b)
  interseption, fscore = fscore_(a,b)
  a = a - interseption 
  b = b - interseption 
  if len(a) == 0 or len(b) == 0: 
      rez = 1 - fscore
  else: 
      penalty = 0
      count_common = 0
      
      #Take words from lists
      for word_a in a:
        for word_b in b:
          hypernyms_a = get_hypernyms(word_a)
          hypernyms_b = get_hypernyms(word_b)
          count_common = len(hypernyms_a.intersection(hypernyms_b))

          if (count_common > 0):
            penalty = penalty + 1  
      
      if penalty > 0:
          prec_penalty = penalty/len(a)
          recall_penalty = penalty/len(b)
          fscore_penalty = 2 * (prec_penalty * recall_penalty) / (prec_penalty + recall_penalty)
      else:
          fscore_penalty = 0
                                                  
      rez = 1 - (fscore + fscore_penalty)   
  
  return rez

        

In [16]:
buff = list(res.items())
dist = np.zeros((len(buff),len(buff)))
for lli,ll in enumerate(buff):
    for rri,rr in enumerate(buff):
        dist[lli,rri]=distance(ll[0],rr[0])

KeyboardInterrupt: ignored

#### Top ten closest articles with fuzzy metrics of titles

In [0]:
@interact(ind=(0,len(buff)-1,1))
def h(ind=0):
    pp = pprint.PrettyPrinter(indent=4)
    print(' '.join(buff[ind][0]))
    pp.pprint([buff[i][0] for i in dist[ind][:].argsort()[1:11]])


interactive(children=(IntSlider(value=0, description='ind', max=995), Output()), _dom_classes=('widget-interac…

In [0]:
@interact(ind=(0,len(buff)-1,1))
def hypernyms(ind=0):
    pp = pprint.PrettyPrinter(indent=4)
    print(' '.join(buff[ind][0]))
    pp.pprint(buff[ind][1])

interactive(children=(IntSlider(value=0, description='ind', max=995), Output()), _dom_classes=('widget-interac…

In [0]:
from scipy import spatial
from numpy import dot
from numpy.linalg import norm
first_list = ['multi', 'photon', 'channel', 'interferometry', 'quantum', 'information', 'processing']
secnd_list = ['quantum', 'photon', 'channel', 'interferometry', 'multi' ]

#first_set = set(map(tuple, first_list))
#secnd_set = set(map(tuple, secnd_list))

first_set = set(first_list)
secnd_set = set(secnd_list) 

#result = 1 - spatial.distance.cosine(first_list, secnd_list)
cos_sim = dot(first_list, secnd_list)/(norm(first_list)*norm(secnd_list))
secnd_set.symmetric_difference(first_set) 
print(result)

In [0]:
buff = list(res.items())
dist = np.zeros((len(buff),len(buff)))
epoch = 0 #just for visualization
for lli,ll in enumerate(buff):
    for rri,rr in enumerate(buff):
        epoch = epoch + 1
        dist[lli,rri]=distance(ll[0],rr[0])
        if (epoch%10000 == 0):
            print('####', epoch)

In [0]:
word_counter = {}

for title in titles:
  for word in title:
    if word in word_counter:
      word_counter[word] += 1
    else:
      word_counter[word] = 1
    popular_words = sorted(word_counter, key = word_counter.get, reverse = True)
  
  top_n = popular_words[:round(len(title))]
  #top_n = popular_words[round(len(title)/2)]  
  #print("length title", round(len(title)/2))
  print(top_n)
  print(round(len(title)))
  word_counter = {}


['much', 'card', 'deck', 'well', 'shuffled']
5
['quantum', 'isometry', 'groups']
3
['computation', 'bounding', 'folkman', 'numbers']
4
['church', 'thesis', 'cubical', 'assemblies']
4
['unconstrained', 'church', 'turing', 'thesis', 'cannot', 'possibly', 'true']
7
['algebraic', 'relaxations', 'hardness', 'results', 'polynomial', 'optimization', 'lyapunov', 'analysis']
8
['resolving', 'complexity', 'fundamental', 'problems', 'computational', 'social', 'choice']
7
['pa', 'instantiationally', 'complete', 'algorithmically', 'incomplete', 'alternative', 'interpretation', 'goedelian', 'incompleteness', 'church', 'thesis', 'links', 'formal', 'logic', 'computability']
15
['numerical', 'modeling', 'fluid', 'flow', 'porous', 'media', 'modelowanie', 'numeryczne', 'transportu', 'plynow', 'przez', 'osrodki', 'porowate']
13
['threebranes', 'theory']
2
['spin', 'torque', 'nano', 'oscillators', 'memristors', 'multi', 'functional', 'nanodevices', 'advanced', 'computing']
10
['effectuses', 'categorical', 

In [0]:
from nltk.corpus import wordnet 
syn = wordnet.synsets('much')[0] 
syn_1= wordnet.synsets('huge')[0]  
print ("Synset much :  ", syn.name()) 
print ("Synset many :  ", syn_1.name()) 

print ("\nSynset abstract term :  ", syn.hypernyms()) 
print ("\nSynset abstract1 term :  ", syn_1.hypernyms()) 

print ("\nSynset specific term :  ",  
       syn.hypernyms()[0].hyponyms()) 

print ("\nSynset specific1 term :  ",  
       syn_1.hypernyms()[0].hyponyms()) 


Synset much :   much.n.01
Synset many :   huge.s.01

Synset abstract term :   [Synset('large_indefinite_quantity.n.01')]

Synset abstract1 term :   []

Synset specific term :   [Synset('barrels.n.01'), Synset('batch.n.02'), Synset('battalion.n.02'), Synset('billyo.n.01'), Synset('boatload.n.01'), Synset('busload.n.01'), Synset('chunk.n.02'), Synset('infinitude.n.01'), Synset('maximum.n.01'), Synset('mile.n.03'), Synset('million.n.02'), Synset('much.n.01'), Synset('myriad.n.01'), Synset('ocean.n.02'), Synset('ream.n.01'), Synset('small_fortune.n.01'), Synset('tons.n.01')]


IndexError: ignored